# PROJE: Hybrid Search RAG – Türkçe Beslenme Asistanı
# DOSYA: FET312_23040301091_GYMPT_2.ipynb

Bu projede, TÜBER 2022 (Türkiye Beslenme Rehberi) PDF'i üzerinde **Hybrid Search (BM25 + Vektör Arama)** kullanan bir RAG (Retrieval-Augmented Generation) altyapısı kuruyoruz.

**Grubumda Üstlendiğim Görev Tanımım:**
Chatbot Mimarisi: Hybrid Search (BM25 + Vector).
Görev: Kullanıcı sorusunu hem anahtar kelime (BM25) hem de anlamsal vektör olarak aratıp, iki sonucu harmanlayan (Reciprocal Rank Fusion) bir yapı kuracak.

**Bu Base Modelin Amacı:**
- Kullanıcının beslenme ile ilgili sorduğu soruya karşılık,
- PDF içinden en alakalı metin parçalarını (chunk) bulmak,
- Bunu yaparken hem anahtar kelime (BM25) hem de anlamsal benzerlik (vektör) aramasını kullanmak,
- İki sonucu **RRF (Reciprocal Rank Fusion)** ile birleştirmek.

**Mimari:**
```
PDF → Chunk'lara böl → Her chunk için:
                         ├── Embedding oluştur → ChromaDB'ye kaydet
                         └── Token'lara ayır → BM25 index'e kaydet

Kullanıcı sorusu → BM25 ara + Vector ara → RRF ile birleştir → Sonuç
```
***NOT:*** base modelin bu hali 450 sayfalık asıl pdf metninin 52 sayfaya düşürülmüş bir özetini kapsar. model tamamlandığında ilgili konu ile alakalı cevap verme konusundaki başarısı önemli ölçüde artacağını öngörüyorum.

## 1- Gerekli Kütüphaneler

In [3]:
!pip install chromadb sentence-transformers rank_bm25 pymupdf --quiet

In [5]:
import torch
import fitz  # PyMuPDF -> PDF okumak için
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from typing import List, Dict, Tuple
import re

print("Kütüphaneler yüklendi.")
print(torch.version.__version__)

Kütüphaneler yüklendi.
2.6.0+cu124


## 2- Çalışma Ortamı Kontrolü (GPU/CPU)

In [6]:
# GPU kullanmamız gerektiği için burada GPU mu yoksa CPU mu kullanıyoruz diye kontrol etmeliyiz.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Çalışma Ortamı: {device.upper()}")

if device == "cuda":
    print(f"GPU Modeli: {torch.cuda.get_device_name(0)}")
    print(f"VRAM Bilgisi: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("UYARI: CPU kullanıyorsunuz. Embedding hesaplamaları yavaş olacaktır!")

Çalışma Ortamı: CUDA
GPU Modeli: NVIDIA GeForce RTX 4060
VRAM Bilgisi: 8.59 GB


## 3- Embedding Modeli ve ChromaDB Kurulumu

In [7]:
# türkçe destekli çok dilli bir embedding modeli kullanıyorum
# internette bakındığım kadarıyla intfloat/multilingual-e5-base modeli türkçe için iyi sonuç veriyor
MODEL_NAME = "intfloat/multilingual-e5-base"

print(f"Model yükleniyor: {MODEL_NAME}")
embed_model = SentenceTransformer(MODEL_NAME, device=device)
print(f"Model başarıyla yüklendi. Device: {device}")

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))

# metin chunk'larını tutulacağı bir koleksiyon oluşturuyoruz
# eğer koleksiyon zaten varsa silip yeniden oluşturuyoruz
try:
    chroma_client.delete_collection(name="tuber_chunks")
except:
    pass

collection = chroma_client.create_collection(
    name="tuber_chunks",
    metadata={"hnsw:space": "cosine"}  # Kosinüs benzerliği kullanıyorum
)

print("ChromaDB koleksiyonu hazır.")

Model yükleniyor: intfloat/multilingual-e5-base
Model başarıyla yüklendi. Device: cuda
ChromaDB koleksiyonu hazır.


## 4- PDF Okuma Fonksiyonu

In [8]:
def read_pdf_text(pdf_path: str) -> str:
    """
    Verilen PDF dosyasındaki tüm sayfaları okuyup
    metin olarak birleştirir.
    """
    doc = fitz.open(pdf_path)
    texts = []
    for page_num, page in enumerate(doc):
        text = page.get_text("text")
        if text.strip():  # Boş sayfaları atlıyorum
            texts.append(text)
    doc.close()
    full_text = "\n".join(texts)
    return full_text


PDF_PATH = "Turkiye_Beslenme_Rehber_TUBER_2022_min.pdf"

raw_text = read_pdf_text(PDF_PATH)

print(f"PDF başarıyla okundu: {PDF_PATH}")
print(f"Toplam karakter sayısı: {len(raw_text):,}")
print("\n--- Metnin ilk 500 karakteri ---\n")
print(raw_text[:500])

PDF başarıyla okundu: Turkiye_Beslenme_Rehber_TUBER_2022_min.pdf
Toplam karakter sayısı: 1,213,140

--- Metnin ilk 500 karakteri ---

TÜRKİYE 
BESLENME 
R E H B E R İ 
( T Ü B E R ) 
2 0 2 2
Ankara - 2022

Eser Adı  : Türkiye Beslenme Rehberi 2022
ISBN  
: 978-975-590-867-0
T.C. Sağlık Bakanlığı Yayın No : 1031
Güncel Baskı, Ankara - 2022
Baskı  
: Hazar Reklam Mat. Yay. Dan. Eği. Kır. İnş. San. ve Tic. Ltd. Şti.
Kazım Karabekir Cad. Kültür Çarşısı No.7/56-57 İskitler - Altındağ / ANKARA
Sertifika No: 48999
Bu yayın; T.C. Sağlık Bakanlığı Halk Sağlığı Genel Müdürlüğü, Sağlıklı Beslenme ve Hareketli Hayat 
Dairesi Başkanlığı ta


## 5- Metni Temizleme ve Chunk'lara Bölme

In [9]:
def clean_text(text: str) -> str:
    """
    Metin içindeki gereksiz boşlukları ve özel karakterleri temizler.
    """
    # birden fazla boşluğu tek boşluğa indir
    text = re.sub(r"\s+", " ", text)
    # tarih formatını temizle (25.11.2024 gibi)
    text = re.sub(r"\d{2}\.\d{2}\.\d{4}", "", text)
    return text.strip()


def split_into_chunks(text: str, max_chars: int = 500, overlap: int = 50) -> List[str]:
    """
    Metni chunk'lara böler:
    - Cümle sonlarına göre (. ! ?) böler
    - max_chars sınırını aşmadan birleştirir
    - overlap ile chunk'lar arası geçiş sağlar
    """
    text = clean_text(text)
    
    # Cümle bazlı ayırma
    sentences = re.split(r"(?<=[.!?])\s+", text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Çok kısa cümleleri atla
        if len(sentence) < 10:
            continue
            
        # mevcut chunk'a eklenebilir mi
        if len(current_chunk) + len(sentence) + 1 <= max_chars:
            current_chunk += " " + sentence
        else:
            # chunk'ı kaydet ve yenisine başla
            if current_chunk.strip():
                chunks.append(current_chunk.strip())
            current_chunk = sentence
    
    # son chunkı ekle
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    
    # çok kısa chunkları filtrele
    chunks = [c for c in chunks if len(c) > 50]
    
    return chunks


# chunklara böl
chunks = split_into_chunks(raw_text, max_chars=500)

print(f"Oluşturulan chunk sayısı: {len(chunks)}")
print(f"Ortalama chunk uzunluğu: {sum(len(c) for c in chunks) // len(chunks)} karakter")
print("\n--- Örnek Chunk (ilk chunk) ---\n")
print(chunks[0])

Oluşturulan chunk sayısı: 2391
Ortalama chunk uzunluğu: 493 karakter

--- Örnek Chunk (ilk chunk) ---

TÜRKİYE BESLENME R E H B E R İ ( T Ü B E R ) 2 0 2 2 Ankara - 2022 Eser Adı : Türkiye Beslenme Rehberi 2022 ISBN : 978-975-590-867-0 T.C. Sağlık Bakanlığı Yayın No : 1031 Güncel Baskı, Ankara - 2022 Baskı : Hazar Reklam Mat. Kazım Karabekir Cad. Kültür Çarşısı No.7/56-57 İskitler - Altındağ / ANKARA Sertifika No: 48999 Bu yayın; T.C.


## 6- Chunk'lara Embedding Oluşturma ve ChromaDB'ye Kaydetme

Bu prefix'ler modelin daha iyi çalışmasını sağlıyor.

In [10]:
# her chunka benzersiz bir id veriyorum
ids = [f"chunk_{i}" for i in range(len(chunks))]

# e5 modeli için passage prefix'i ekliyoruz
passages = [f"passage: {chunk}" for chunk in chunks]

print("Embedding'ler hesaplanıyor...")

embeddings = embed_model.encode(
    passages,
    convert_to_tensor=False,
    normalize_embeddings=True,
    show_progress_bar=True
)

print(f"Embedding boyutu: {embeddings[0].shape}")

# ChromaDB koleksiyonuna eklenmesi
collection.add(
    documents=chunks,
    embeddings=[emb.tolist() for emb in embeddings],
    ids=ids,
    metadatas=[{"chunk_id": i, "length": len(chunks[i])} for i in range(len(chunks))]
)

print(f"\nChromaDB'ye {len(chunks)} chunk eklendi.")

Embedding'ler hesaplanıyor...


Batches: 100%|██████████| 75/75 [00:11<00:00,  6.44it/s]


Embedding boyutu: (768,)

ChromaDB'ye 2391 chunk eklendi.


## 7- BM25 Index Oluşturma (Anahtar Kelime Araması)

In [11]:
def tokenize_turkish(text: str) -> List[str]:
    """
    Türkçe için basit tokenizer:
    - Küçük harfe çevirir
    - Noktalama işaretlerini temizler
    - Kelimelere böler
    - Çok kısa kelimeleri (2 karakterden az) atar
    """
    text = text.lower()
    # Noktalama işaretlerini kaldır
    text = re.sub(r"[^\w\sğüşıöçĞÜŞİÖÇ]", " ", text)
    # Kelimelere böl
    tokens = text.split()
    # Çok kısa kelimeleri filtrele
    tokens = [t for t in tokens if len(t) > 2]
    return tokens

tokenized_corpus = [tokenize_turkish(chunk) for chunk in chunks]

bm25 = BM25Okapi(tokenized_corpus)

print(f"BM25 index hazır.")
print(f"Toplam doküman sayısı: {len(tokenized_corpus)}")
print(f"Örnek tokenlar (ilk chunk): {tokenized_corpus[0][:10]}...")

BM25 index hazır.
Toplam doküman sayısı: 2391
Örnek tokenlar (ilk chunk): ['türki', 'beslenme', 'ankara', '2022', 'eser', 'adı', 'türkiye', 'beslenme', 'rehberi', '2022']...


## 8- Arama Fonksiyonları (BM25 ve Vektör)

In [12]:
def bm25_search(query: str, top_k: int = 5) -> List[Tuple[int, float]]:
    """
    BM25 ile anahtar kelime araması yapar.
    
    Returns:
        List of (chunk_index, score) tuples
    """
    tokenized_query = tokenize_turkish(query)
    scores = bm25.get_scores(tokenized_query)
    
    # Index ve skor çiftlerini sırala
    ranked = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:top_k]
    return ranked


def vector_search(query: str, top_k: int = 5) -> List[Tuple[int, float]]:
    """
    ChromaDB ile vektör (semantic) araması yapar.
    
    Returns:
        List of (chunk_index, distance) tuples
    """
    # e5 modeli için query prefixinin eklenmesi
    q_text = f"query: {query}"
    q_emb = embed_model.encode(
        [q_text],
        convert_to_tensor=False,
        normalize_embeddings=True
    )
    
    results = collection.query(
        query_embeddings=q_emb.tolist(),
        n_results=top_k,
        include=["distances"]
    )
    
    # Sonuçları parse et
    result_ids = results["ids"][0]
    distances = results["distances"][0]
    
    # (index, distance) çiftleri döndür
    parsed = []
    for cid, dist in zip(result_ids, distances):
        idx = int(cid.split("_")[1])
        parsed.append((idx, dist))
    
    return parsed

## 9- RRF (Reciprocal Rank Fusion) Fonksiyonu

İki farklı arama sonucunu birleştirmek için RRF kullanıyoruz

**Formül:** `score(d) = Σ 1 / (k + rank(d))`

- `k` sabit bir parametre (genelde 60)
- Her dokümana, her listede bulunduğu sıraya göre skor veriliyor
- Skorlar toplanıp en yüksekten sıralanıyor

In [13]:
def rrf_fusion(
    bm25_results: List[Tuple[int, float]], 
    vector_results: List[Tuple[int, float]], 
    k: int = 60
) -> List[Tuple[int, float]]:
    """
    Reciprocal Rank Fusion (RRF) ile iki arama sonucunu birleştirir.
    
    Args:
        bm25_results: BM25 sonuçları [(index, score), ...]
        vector_results: Vektör arama sonuçları [(index, distance), ...]
        k: RRF parametresi (default: 60)
    
    Returns:
        Birleştirilmiş ve sıralanmış sonuçlar [(index, rrf_score), ...]
    """
    scores = {}
    
    # BM25 sonuçlarından skor hesapla
    for rank, (idx, _) in enumerate(bm25_results):
        scores[idx] = scores.get(idx, 0) + 1 / (k + rank + 1)
    
    # Vektör sonuçlarından skor hesapla
    for rank, (idx, _) in enumerate(vector_results):
        scores[idx] = scores.get(idx, 0) + 1 / (k + rank + 1)
    
    # Skora göre sırala
    fused = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    return fused

## 10- Hybrid Search Ana Fonksiyonu

In [14]:
def hybrid_search(
    query: str, 
    top_k_each: int = 5, 
    final_k: int = 3,
    verbose: bool = True
) -> Dict:
    """
    Hybrid Search: BM25 + Vektör + RRF
    
    Args:
        query: Kullanıcı sorusu
        top_k_each: Her arama yönteminden alınacak sonuç sayısı
        final_k: Final olarak döndürülecek chunk sayısı
        verbose: Detaylı çıktı göster
    
    Returns:
        Dict containing query, results, and intermediate data
    """
    # 1) BM25 araması
    bm25_results = bm25_search(query, top_k=top_k_each)
    
    # 2) Vektör araması
    vector_results = vector_search(query, top_k=top_k_each)
    
    # 3) RRF ile birleştir
    fused_results = rrf_fusion(bm25_results, vector_results, k=60)
    
    # 4) En iyi sonuçları al
    best_results = fused_results[:final_k]
    
    # 5) Chunk metinlerini ekle
    results_with_text = [
        {
            "chunk_id": idx,
            "rrf_score": score,
            "text": chunks[idx]
        }
        for idx, score in best_results
    ]
    
    output = {
        "query": query,
        "bm25_ids": [idx for idx, _ in bm25_results],
        "vector_ids": [idx for idx, _ in vector_results],
        "fused_ids": [idx for idx, _ in fused_results],
        "results": results_with_text
    }
    
    if verbose:
        print(f"BM25 Sonuçları (top {top_k_each}): {output['bm25_ids']}")
        print(f"Vektör Sonuçları (top {top_k_each}): {output['vector_ids']}")
        print(f"RRF Birleşim (top 5): {output['fused_ids'][:5]}")
    
    return output

## 11- Sistemi Test Etme

In [15]:
# TÜBER 2022 içeriğine uygun test soruları
test_questions = [
    "Günde kaç porsiyon sebze yemeliyim?",
    "Protein ihtiyacı nasıl karşılanır?",
    "Çocuklar için süt tüketimi ne kadar olmalı?",
    "Obezite nedir ve BKI nasıl hesaplanır?",
    "Sağlıklı beslenme tabağı nasıl olmalı?"
]

print("=" * 80)
print("HYBRID SEARCH TEST SONUÇLARI")
print("=" * 80)

for q in test_questions:
    print(f"\n{'='*80}")
    print(f"SORU: {q}")
    print("-" * 40)
    
    result = hybrid_search(q, top_k_each=5, final_k=3, verbose=True)
    
    print("\n--- EN İYİ SONUÇLAR ---")
    for i, res in enumerate(result["results"], 1):
        print(f"\n[{i}] Chunk #{res['chunk_id']} (RRF Skor: {res['rrf_score']:.4f})")
        print(f"{res['text'][:300]}...")

HYBRID SEARCH TEST SONUÇLARI

SORU: Günde kaç porsiyon sebze yemeliyim?
----------------------------------------
BM25 Sonuçları (top 5): [345, 2081, 579, 1172, 614]
Vektör Sonuçları (top 5): [692, 691, 345, 564, 346]
RRF Birleşim (top 5): [345, 692, 2081, 691, 579]

--- EN İYİ SONUÇLAR ---

[1] Chunk #345 (RRF Skor: 0.0323)
Toplam taze meyve tüketimi erkeklerde ortalama 162.7 g, kadınlarda 155.0 g, toplam kuru meyve tüketimi ise erkeklerde 2.2 g, kadınlarda 3.1 g olup, tüm bireylerde kuru meyve tüketimi ortalama 2.6 g’dır. Günde en az 5 porsiyon (en az 400 g/gün) sebze ve meyve tüketilmeli, bunlardan en az 2.5- 4 porsi...

[2] Chunk #692 (RRF Skor: 0.0164)
Ek 3.1.1’de ise “ Türkiye için Enerji Düzeylerine (1000-3200 kkal) göre Önerilen Beslenme Örüntüleri”ne göre tüketilmesi önerilen sebze ve meyve porsiyon miktarları verilmiştir. | 73 | Türkiye Beslenme Rehberi | 2022 • Her gün en az 400 g (en az 5 porsiyon) taze sebze ve meyve tüketilmelidir. • Beş ...

[3] Chunk #2081 (RRF Skor: 0.0

## 12- Basit Soru-Cevap Fonksiyonu (Opsiyonel)

Bu fonksiyon, hybrid search sonuçlarını birleştirip kullanıcıya sunar.
**Not:** Bu base versiyonda LLM yok, sadece retrieval sonuçları gösteriliyor.

In [16]:
def answer_question(query: str, top_k_each: int = 5, final_k: int = 3):
    """
    Kullanıcı sorusuna hybrid search ile cevap bulur.
    
    Not: Bu base versiyonda LLM entegrasyonu yok.
    Sadece en alakalı chunk'ları döndürüyor.
    """
    print("\n" + "=" * 80)
    print("🔍 SORU:", query)
    print("=" * 80)
    
    result = hybrid_search(query, top_k_each=top_k_each, final_k=final_k, verbose=False)
    
    print("\n📚 TÜBER 2022'DEN BULUNAN İLGİLİ BİLGİLER:\n")
    
    for i, res in enumerate(result["results"], 1):
        print(f"--- Kaynak {i} (Chunk #{res['chunk_id']}) ---")
        print(res["text"])
        print()
    
    return result


# Örnek kullanım
answer_question("Günlük su tüketimi ne kadar olmalı?")


🔍 SORU: Günlük su tüketimi ne kadar olmalı?

📚 TÜBER 2022'DEN BULUNAN İLGİLİ BİLGİLER:

--- Kaynak 1 (Chunk #373) ---
Günlük yağ alım miktarı kadar, türü ve bu türler arasındaki oran da önem taşır. Öneri: Yağların günlük tüketim miktarları Diyetimizdeki yiyeceklerin yapısında aldığımız yağa “görünmez”, doğrudan yağ olarak alınana da “görünür” yağ denilir. Ülkemizde, kişi başına günlük görünür yağ tüketimi; TBSA 2017 verilerine göre katı yağ tüketimi 15 ve üzeri yaş grubu erkeklerde ortalama 13.7 g, kadınlarda 9.1 gramdır.

--- Kaynak 2 (Chunk #398) ---
Ancak suyla birlikte sodyum, potasyum gibi mineraller de kaybedildiği zaman susama merkezi uyarılmaz ve susama duygusu gelişmez. TBSA 2017’ye göre bireylerin günlük su tüketim ortalaması 1594.3±968.99 mL (erkeklerin 1766.4±1039.56 mL, kadınların 1423.8±860.38 mL)’dir. Çeşitli yaş gruplarına göre su tüketim önerisi Ek 1.4.1’de gösterilmiştir Su toksisitesi: Su için tolere edilebilir üst alım düzeyi yoktur. Fazla alınan su, böbrekler ve t

{'query': 'Günlük su tüketimi ne kadar olmalı?',
 'bm25_ids': [373, 1002, 344, 1352, 405],
 'vector_ids': [398, 393, 401, 261, 1386],
 'fused_ids': [373, 398, 1002, 393, 344, 401, 1352, 261, 405, 1386],
 'results': [{'chunk_id': 373,
   'rrf_score': 0.01639344262295082,
   'text': 'Günlük yağ alım miktarı kadar, türü ve bu türler arasındaki oran da önem taşır. Öneri: Yağların günlük tüketim miktarları Diyetimizdeki yiyeceklerin yapısında aldığımız yağa “görünmez”, doğrudan yağ olarak alınana da “görünür” yağ denilir. Ülkemizde, kişi başına günlük görünür yağ tüketimi; TBSA 2017 verilerine göre katı yağ tüketimi 15 ve üzeri yaş grubu erkeklerde ortalama 13.7 g, kadınlarda 9.1 gramdır.'},
  {'chunk_id': 398,
   'rrf_score': 0.01639344262295082,
   'text': 'Ancak suyla birlikte sodyum, potasyum gibi mineraller de kaybedildiği zaman susama merkezi uyarılmaz ve susama duygusu gelişmez. TBSA 2017’ye göre bireylerin günlük su tüketim ortalaması 1594.3±968.99 mL (erkeklerin 1766.4±1039.56 mL, 

## 13- İnteraktif Test (Opsiyonel)

In [29]:
# İnteraktif test için bu hücreyi çalıştır
# Çıkmak için 'q' veya 'çık' yaz

print("\n" + "="*80)
print("🥗 TÜBER 2022 - Hybrid Search RAG Sistemi")
print("="*80)
print("Beslenme ile ilgili sorularınızı yazın. Çıkmak için 'q' yazın.\n")

while True:
    user_input = input("\n📝 Sorunuz: ").strip()
    
    if user_input.lower() in ['q', 'quit', 'çık', 'exit', '']:
        print("\n👋 Görüşmek üzere!")
        break
    
    answer_question(user_input)


🥗 TÜBER 2022 - Hybrid Search RAG Sistemi
Beslenme ile ilgili sorularınızı yazın. Çıkmak için 'q' yazın.


🔍 SORU: sağlıklı yetişkin bir erkeğin günde kaç lt su tüketmesi uygundur

📚 TÜBER 2022'DEN BULUNAN İLGİLİ BİLGİLER:

--- Kaynak 1 (Chunk #787) ---
Kolesterolden zengin bir besin olmasına karşın yumurtanın kan kolesterolünü yükseltici etkisi yağlı et ve süt ve ürünlerinden daha az olduğu için sağlıklı bireylerin kolesterol içeriğinden dolayı yumurta tüketiminden kaçınmasına gerek yoktur. Sağlıklı bireyler tam yumurtayı günde bir adet, kalp damar hastalıkları olanlar da haftada 2-3 kez bir adet tüketebilir.

--- Kaynak 2 (Chunk #261) ---
Vücut suyu bebeklik çağında %80 iken yaş ilerledikçe azalarak ileri yaşlarda %60-50’lere düşer. Vücut suyunun %10 kaybı ölümle sonuçlanır. Avrupa Gıda Güvenliği Otoritesi (EFSA) 2010 tarafından yetişkin erkeklerin günde 2.5 litre ve kadınların 2.0 litre (1 mL/kkal) olarak önerilen toplam sıvı alım miktarının %80’inin su ve içeceklerden (8- 10 bardak

KeyboardInterrupt: Interrupted by user

---

## Özet

1.  TÜBER 2022 PDF'ini okuduk ve chunk'lara böldük
2.  Her chunk için embedding oluşturduk (multilingual-e5-base)
3.  ChromaDB'ye vektörleri kaydettik
4.  BM25 index'i oluşturduk
5.  Hybrid Search (BM25 + Vektör + RRF) sistemi kurduk
6.  Test ettik